# Market Regime Tuner

**Regimes:** Green=Up, Yellow=Range, Red=Down

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.collections import LineCollection
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from pathlib import Path
from datetime import datetime
import json
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, str(Path('..').resolve()))
from src.config import FEATURES_DIR, LABELS_DIR

plt.style.use('dark_background')
SYMBOL = 'BTCUSDT'
DATA = {'df': None, 'df_display': None, 'interval': None}

available = sorted([f.stem.split('_')[1] for f in FEATURES_DIR.glob(f'{SYMBOL}_*_features.parquet')])
default_int = '1h' if '1h' in available else available[0]
print(f"Available: {available}, Default: {default_int}")

In [ ]:
def detect_regimes(df, params):
    df = df.copy()
    n = len(df)
    bp = df['bullish_pct_sma'].values.copy()
    pp = df['price_position'].values.copy()
    sp = df['spread_pct'].values.copy()
    
    if params['use_ema_smooth'] and params['ema_span'] > 1:
        bp = pd.Series(bp).ewm(span=params['ema_span']).mean().values
        pp = pd.Series(pp).ewm(span=params['ema_span']).mean().values
        sp = pd.Series(sp).ewm(span=params['ema_span']).mean().values
    
    raw = np.zeros(n, dtype=int)
    for i in range(n):
        b, p, s = bp[i], pp[i], sp[i]
        if np.isnan(b): b = 0.5
        if np.isnan(p): p = 0.5
        if np.isnan(s): s = 0
        if b >= params['bullish_threshold_up'] and p >= params['price_pos_up'] and s >= params['spread_min']:
            raw[i] = 1
        elif b <= params['bullish_threshold_down'] and p <= params['price_pos_down'] and s >= params['spread_min']:
            raw[i] = 2
    
    smooth = np.zeros(n, dtype=int)
    curr, pend, pend_ct = 0, None, 0
    for i in range(n):
        r = raw[i]
        if r == curr:
            pend, pend_ct = None, 0
        else:
            pend_ct = pend_ct + 1 if pend == r else 1
            pend = r
            req = params['min_bars'] + (params['hysteresis'] if curr != 0 and r == 0 else 0)
            if pend_ct >= req:
                curr, pend, pend_ct = r, None, 0
        smooth[i] = curr
    
    df['regime'], df['raw_regime'] = smooth, raw
    return df

In [ ]:
# Widgets
int_dd = widgets.Dropdown(options=available, value=default_int, description='Interval:')
bars_dd = widgets.Dropdown(options=[('500',500),('1000',1000),('2000',2000),('5000',5000),('All',-1)], value=2000, description='Bars:')
pos_sl = widgets.IntSlider(value=100, min=0, max=100, description='Position:', continuous_update=False)
load_btn = widgets.Button(description='Load', button_style='primary')
status = widgets.HTML('<i>No data</i>')

bu_sl = widgets.FloatSlider(value=0.65, min=0.5, max=0.95, step=0.05, description='Bull Up:', continuous_update=False)
bd_sl = widgets.FloatSlider(value=0.35, min=0.05, max=0.5, step=0.05, description='Bull Down:', continuous_update=False)
pu_sl = widgets.FloatSlider(value=0.65, min=0.5, max=0.95, step=0.05, description='Price Up:', continuous_update=False)
pd_sl = widgets.FloatSlider(value=0.35, min=0.05, max=0.5, step=0.05, description='Price Down:', continuous_update=False)
sp_sl = widgets.FloatSlider(value=0.3, min=0.0, max=2.0, step=0.1, description='Spread:', continuous_update=False)
mb_sl = widgets.IntSlider(value=3, min=1, max=20, description='Min Bars:', continuous_update=False)
hy_sl = widgets.IntSlider(value=2, min=0, max=10, description='Hysteresis:', continuous_update=False)
ema_cb = widgets.Checkbox(value=True, description='EMA')
es_sl = widgets.IntSlider(value=5, min=2, max=20, description='EMA Span:', continuous_update=False)
ma_cb = widgets.Checkbox(value=True, description='Show MAs')
raw_cb = widgets.Checkbox(value=False, description='Raw')
upd_btn = widgets.Button(description='Update', button_style='success')
chart_out = widgets.Output()
stats_out = widgets.HTML()

In [ ]:
def load_data(b=None):
    try:
        df = pd.read_parquet(FEATURES_DIR / f'{SYMBOL}_{int_dd.value}_features.parquet')
        df['open_time'] = pd.to_datetime(df['open_time'])
        df = df.sort_values('open_time').reset_index(drop=True)
        DATA['df'], DATA['interval'] = df, int_dd.value
        n = bars_dd.value if bars_dd.value != -1 else len(df)
        start = int(max(0, len(df)-n) * pos_sl.value/100)
        DATA['df_display'] = df.iloc[start:start+n].reset_index(drop=True)
        status.value = f'<b style="color:lime">Loaded {len(DATA["df_display"]):,} bars</b>'
    except Exception as e:
        status.value = f'<b style="color:red">{e}</b>'

load_btn.on_click(load_data)

In [ ]:
def get_params():
    return {'bullish_threshold_up': bu_sl.value, 'bullish_threshold_down': bd_sl.value,
            'price_pos_up': pu_sl.value, 'price_pos_down': pd_sl.value, 'spread_min': sp_sl.value,
            'min_bars': mb_sl.value, 'hysteresis': hy_sl.value, 'use_ema_smooth': ema_cb.value, 'ema_span': es_sl.value}

def update_chart(b=None):
    if DATA['df_display'] is None:
        with chart_out: clear_output(); print("Load data first!")
        return
    
    df = detect_regimes(DATA['df_display'], get_params())
    DATA['df_with_regime'] = df
    
    fig, ax = plt.subplots(figsize=(16, 9), facecolor='#1a1a1a')
    ax.set_facecolor('#1a1a1a')
    times = np.arange(len(df))
    
    # Backgrounds
    bg = {0:'#4a4a00', 1:'#004a00', 2:'#4a0000'}
    rcol = 'raw_regime' if raw_cb.value else 'regime'
    cr, bs = df.iloc[0][rcol], 0
    for i in range(1, len(df)):
        r = df.iloc[i][rcol]
        if r != cr or i == len(df)-1:
            ax.axvspan(bs, i if r != cr else i+1, alpha=0.5, color=bg[cr], zorder=0)
            cr, bs = r, i
    
    # MAs
    if ma_cb.value:
        for m in range(5, 37):
            c = f'ma{m}_sma'
            if c not in df.columns: continue
            v = df[c].values
            sl = np.zeros(len(v)); sl[1:] = v[1:] - v[:-1]; sl[0] = sl[1] if len(sl)>1 else 0
            pts = np.array([times, v]).T.reshape(-1,1,2)
            segs = np.concatenate([pts[:-1], pts[1:]], axis=1)
            cols = np.where(sl[1:] >= 0, '#00ff00', '#ff0000')
            ax.add_collection(LineCollection(segs, colors=cols, linewidths=1.2, alpha=0.7, zorder=1))
    
    # Candles
    up, dn = df[df['close']>=df['open']], df[df['close']<df['open']]
    ax.bar(up.index, up['close']-up['open'], 0.6, bottom=up['open'], color='#26a69a', zorder=2)
    ax.bar(up.index, up['high']-up['close'], 0.1, bottom=up['close'], color='#26a69a', zorder=2)
    ax.bar(up.index, up['low']-up['open'], 0.1, bottom=up['open'], color='#26a69a', zorder=2)
    ax.bar(dn.index, dn['close']-dn['open'], 0.6, bottom=dn['open'], color='#ef5350', zorder=2)
    ax.bar(dn.index, dn['high']-dn['open'], 0.1, bottom=dn['open'], color='#ef5350', zorder=2)
    ax.bar(dn.index, dn['low']-dn['close'], 0.1, bottom=dn['close'], color='#ef5350', zorder=2)
    
    ax.set_xlim(0, len(df)); ax.set_ylim(df['low'].min()*0.998, df['high'].max()*1.002)
    st = max(1, len(df)//10); ax.set_xticks(range(0,len(df),st))
    ax.set_xticklabels([pd.Timestamp(df['open_time'].iloc[i]).strftime('%m/%d') for i in range(0,len(df),st)], rotation=45, color='white')
    ax.tick_params(colors='white'); ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x,p: f'${x:,.0f}'))
    ax.set_title(f'BTCUSDT {DATA["interval"]}', color='white', fontsize=14); ax.grid(True, alpha=0.2)
    ax.legend(handles=[mpatches.Patch(color='#004a00',alpha=0.6,label='Up'), mpatches.Patch(color='#4a4a00',alpha=0.6,label='Range'),
                       mpatches.Patch(color='#4a0000',alpha=0.6,label='Down'), plt.Line2D([0],[0],color='#00ff00',lw=2,label='MA+'),
                       plt.Line2D([0],[0],color='#ff0000',lw=2,label='MA-')], loc='upper left', facecolor='#2a2a2a')
    plt.tight_layout()
    with chart_out: clear_output(); plt.show()
    plt.close(fig)
    
    ct = df['regime'].value_counts(); ch = (df['regime']!=df['regime'].shift(1)).sum()-1
    stats_out.value = f'<div style="background:#1a1a1a;padding:10px;color:white;border:1px solid #444"><b style="color:#0f0">Up:</b>{ct.get(1,0):,} | <b style="color:#ff0">Range:</b>{ct.get(0,0):,} | <b style="color:#f00">Down:</b>{ct.get(2,0):,} | <b>Changes:</b>{ch}</div>'

upd_btn.on_click(update_chart)

In [ ]:
exp_out = widgets.Output()
def export(b):
    with exp_out:
        clear_output()
        if DATA['df'] is None: print("Load first!"); return
        df = detect_regimes(DATA['df'], get_params())
        p = FEATURES_DIR / f"{SYMBOL}_{DATA['interval']}_features_labeled.parquet"
        df.to_parquet(p, index=False)
        ct = df['regime'].value_counts()
        print(f"Saved {len(df):,} bars to {p}\nUp:{ct.get(1,0):,} Range:{ct.get(0,0):,} Down:{ct.get(2,0):,}")

exp_btn = widgets.Button(description='Export Full', button_style='success')
exp_btn.on_click(export)

In [ ]:
display(widgets.VBox([
    widgets.HTML('<h2>Market Regime Tuner</h2>'),
    widgets.HBox([int_dd, bars_dd, pos_sl, load_btn, status]),
    widgets.HBox([widgets.VBox([bu_sl,bd_sl,pu_sl,pd_sl,sp_sl]), widgets.VBox([mb_sl,hy_sl,ema_cb,es_sl])]),
    widgets.HBox([ma_cb, raw_cb, upd_btn]),
    chart_out, stats_out,
    widgets.HBox([exp_btn, exp_out])
]))

load_data()
if DATA['df_display'] is not None: update_chart()